In [ ]:
%run env_setup.py
%matplotlib inline
import os
from importlib import reload

In [ ]:
import lessdeep as ld
import numpy as np

In [ ]:
#img_file = ld.utils.download_file('http://farm2.static.flickr.com/1422/987713710_f801e72921.jpg')
img_file = ld.utils.download_file('http://epmgsenior.media.clients.ellingtoncms.com/img/photos/2012/04/30/p_2_goldfinch_2_t750x550.jpg?d885fc46c41745b3b5de550c70336c1b382931d2')

In [ ]:
from matplotlib import pyplot as plt
from PIL import Image
img = np.array(Image.open(img_file), dtype=np.float32)
plt.imshow(img.astype(np.uint8))
plt.show()

In [ ]:
van_img = Image.open(ld.utils.download_file('https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/1280px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg'))
van_img = np.array(van_img.resize(np.divide(van_img.size, 3).astype(int)), dtype=np.float32)
plt.imshow(van_img.astype(np.uint8))
plt.show()

In [ ]:
%pdb

In [ ]:
from lessdeep.model.vgg16n import Vgg16N
import keras

vgg_avg=Vgg16N(include_top=False)
vgg_avg_layers = []
for l in vgg_avg.model.layers:
    if isinstance(l, keras.layers.MaxPool2D):
        vgg_avg_layers.append(keras.layers.AveragePooling2D((2, 2), strides=(2, 2)))
    else:
        vgg_avg_layers.append(l)
vgg_avg.model = ld.utils.clone_model(vgg_avg_layers)
vgg_avg.compile()
vgg_avg.model.summary()

In [ ]:
conv_idx = [i for i, l in enumerate(vgg_avg.model.layers) if isinstance(l, keras.layers.Conv2D)]

In [ ]:
conv_idx

In [ ]:
from keras.applications.imagenet_utils import preprocess_input
img_input = preprocess_input(img)[None]
van_img_input = preprocess_input(van_img)[None]

In [ ]:
import keras.backend as K
def reverse_input(x):
    data_format = K.image_data_format()
    if data_format == 'channels_first':
        if x.ndim == 3:
            # 'BGR'->'RGB'
            x = x[::-1, ...]
            # Zero-center by mean pixel
            x[0, :, :] += 123.68
            x[1, :, :] += 116.779
            x[2, :, :] += 103.939
        else:
            x = x[:, ::-1, ...]
            x[:, 0, :, :] += 123.68
            x[:, 1, :, :] += 116.779
            x[:, 2, :, :] += 103.939
    else:
        # 'BGR'->'RGB'
        x = x[..., ::-1]
        # Zero-center by mean pixel
        x[..., 0] += 123.68
        x[..., 1] += 116.779
        x[..., 2] += 103.939
    return x

In [ ]:
plt.imshow(reverse_input(img_input[0]).astype(np.uint8))
plt.show()

In [ ]:
layer = vgg_avg.model.layers[conv_idx[-6]].output
layer_model = keras.Model(vgg_avg.model.input, layer)

In [ ]:
tgt = K.variable(layer_model.predict(img_input))

In [ ]:
loss = K.mean(keras.metrics.mse(tgt, layer))
grad = K.gradients(loss, vgg_avg.model.input)

In [ ]:
class Runner(object):
    def __init__(self, f, in_shape):
        self._f, self.shape = f, in_shape
    def loss(self, x):
        loss, self._grad = self._f([x.reshape(self.shape)])
        return loss.astype(np.float64)
    def grad(self, x):
        return self._grad.astype(np.float64).flatten()
runner = Runner(K.function([vgg_avg.model.input], [loss]+grad), img_input.shape)

In [ ]:
from scipy.optimize import fmin_l_bfgs_b

def learn_image(run_obj, x, niter):
    his = []
    for i in range(niter):
        x, f, d = fmin_l_bfgs_b(run_obj.loss, x.flatten(), fprime=run_obj.grad, maxiter=20)
        his.append(x.reshape(run_obj.shape).copy())
        print('Iter {0}: loss {1}'.format(i, f))
    return his

In [ ]:
rand_img = np.array(np.random.uniform(-2.5, 2.5, img_input.shape).astype(np.float32)/100)
#rand_img[0]
import matplotlib
plt.imshow(rand_img[0][:,:,0])
plt.show()

In [ ]:
his = learn_image(runner, rand_img, niter=20)

In [ ]:
plt.imshow(reverse_input(his[-1][0]).astype(np.uint8))

In [ ]:
from IPython.display import HTML
from matplotlib import animation, rc

In [ ]:
fig, ax = plt.subplots()
def animate(i):
    ax.imshow(reverse_input(his[-1][0]).astype(np.uint8))
anim = animation.FuncAnimation(fig, animate, frames=10, interval=200)
HTML(anim.to_html5_video())

## Creating Style

In [ ]:
plt.imshow(reverse_input(van_img_input[0]).astype(np.uint8))
plt.show()

In [ ]:
style_layers = [vgg_avg.model.layers[conv_idx[i]].output for i in range(1, len(conv_idx), 3)]
style_model = keras.Model(vgg_avg.model.input, style_layers)
style_tgts = [K.variable(v) for v in style_model.predict(van_img_input)]

In [ ]:
def gram_matrix(x):
    tmp = K.permute_dimensions(x, (2, 0, 1))  # convert to (depth, y, x)
    features = K.batch_flatten(tmp)  # keep depth, flatten each feature into one row
    return K.dot(features, K.transpose(features)) / x.get_shape().num_elements()

In [ ]:
def style_loss(x, tgt):
    return K.mean(keras.metrics.mse(gram_matrix(x), gram_matrix(tgt)))

In [ ]:
x=style_layers[0][0]
tmp = K.permute_dimensions(x, (2, 0, 1))  # convert to (depth, y, x)
tmp
features = K.batch_flatten(tmp)  # keep depth, flatten each feature into one row
tmp, features, x.get_shape()
#K.dot(features, K.transpose(features)) / x.get_shape().num_elements()

In [ ]:
loss = sum([style_loss(l[0], t[0]) for l, t in zip(style_layers, style_tgts)])
grad = K.gradients(loss, [vgg_avg.model.input])
style_runner = Runner(K.function([vgg_avg.model.input], [loss]+grad), van_img_input.shape)